https://gitlab.com/pqc-hqc/hqc/

In [21]:
def schoolbook_mul64(a, b):
    """
    Perform GF(2) schoolbook multiplication of two n-limb (64-bit) binary polynomials.

    Args:
        a, b: integers in range [0, 2**64), representing 64-bit binary polynomials in GF(2)

    Returns:
        r: list of 2*n integers, representing the product polynomial (2n limbs)
    """

    a &= (1 << 64) - 1
    b &= (1 << 64) - 1
    low = 0
    high = 0
    for bit in range(64):
        if (a >> bit) & 1:
            if bit == 0:
                low ^= b
            else:
                low ^= (b << bit)
                high ^= (b >> (64 - bit))
    mask64 = (1 << 64) - 1
    return [(low & mask64), (high & mask64)]


In [2]:
# Multiply two 128-bit polynomials
a = 0x0000000000000011  # 128-bit input A
b = 0x0000000000000011  # 128-bit input B

result = schoolbook_mul64(a, b)
print("Result limbs (low to high):")
for i, limb in enumerate(result):
    print(f"r[{i}] = 0x{limb:016x}")

Result limbs (low to high):
r[0] = 0x0000000000000101
r[1] = 0x0000000000000000


In [22]:
# Multiply two 128-bit polynomials
a = 0x0000000000000101  # 128-bit input A
b = 0x0000000000000101  # 128-bit input B

result = schoolbook_mul64(a, b)
print("Result limbs (low to high):")
for i, limb in enumerate(result):
    print(f"r[{i}] = 0x{limb:016x}")

Result limbs (low to high):
r[0] = 0x0000000000010001
r[1] = 0x0000000000000000


In [42]:
import galois

GF2 = galois.GF(2)

def gf2_mul_to_int(a: int, b: int) -> int:
    pa = galois.Poly.Int(a, field=GF2)  # Correctly interprets bits as poly coeffs
    pb = galois.Poly.Int(b, field=GF2)
    pc = pa * pb
    return int(pc)

# Test
result = gf2_mul_to_int(a, b)  # 8 = x^3, so x^3 * x^3 = x^6 = 64
print(f"{a} * {b} in GF(2) = {result}")  # Output: 17

12354532453 * 12543532453 in GF(2) = 78822079870137848529


In [41]:
# Multiply two 128-bit polynomials
# a = 123  # 128-bit input A
# b = 123  # 128-bit input B
a = 12354532453      # x^17668 + 12345
b = 12543532453  # sparse polynomial

result = schoolbook_mul64(a, b)
print("Result limbs (low to high):", result)

Result limbs (low to high): [5035103575299642065, 4]


In [39]:
def combine_limbs(lo, hi):
    """Combine two 64-bit limbs into a Python int."""
    return lo | (hi << 64)

def split_128_to_64(x):
    """Split a 128-bit integer into two 64-bit limbs [low, high]."""
    x &= (1 << 128) - 1
    return [x & ((1 << 64) - 1), (x >> 64) & ((1 << 64) - 1)]

def add_128(a_lo, a_hi, b_lo, b_hi):
    """GF(2) addition (XOR) of two 128-bit numbers given as limbs."""
    return (a_lo ^ b_lo, a_hi ^ b_hi)

def karatsuba_mul128(a, b):
    """
    Multiply two 128-bit integers as GF(2) polynomials using Karatsuba.
    Returns 256-bit result as list of 4 64-bit limbs: [L0, L1, L2, L3]
    """
    # Split inputs into 64-bit limbs (low, high)
    a0, a1 = split_128_to_64(a)
    b0, b1 = split_128_to_64(b)

    # z0 = a0 * b0
    z0_lo, z0_hi = schoolbook_mul64(a0, b0)

    # z2 = a1 * b1
    z2_lo, z2_hi = schoolbook_mul64(a1, b1)

    # Compute (a0 + a1) and (b0 + b1) in GF(2) → XOR
    a0a1_lo, a0a1_hi = a0 ^ a1, 0  # since a0, a1 are 64-bit
    b0b1_lo, b0b1_hi = b0 ^ b1, 0

    # m1 = (a0 + a1) * (b0 + b1)
    m1_lo, m1_hi = schoolbook_mul64(a0a1_lo, b0b1_lo)

    # z1 = m1 - z0 - z2 → in GF(2): z1 = m1 XOR z0 XOR z2
    # But z0 and z2 are 128-bit, m1 is 128-bit
    z1_lo = m1_lo ^ z0_lo ^ z2_lo
    z1_hi = m1_hi ^ z0_hi ^ z2_hi

    # Now build the final 256-bit result:
    # result = z2 * x^128 + z1 * x^64 + z0
    # Represent as 4 limbs: [L0, L1, L2, L3]
    L0 = z0_lo
    L1 = z0_hi
    L2 = z1_lo
    L3 = z1_hi
    L4 = z2_lo
    L5 = z2_hi

    # Now add (XOR) overlapping limbs due to shifts:
    # z0:      [L0, L1,  0,  0,  0,  0]
    # z1<<64:  [ 0, L2, L3,  0,  0,  0]
    # z2<<128: [ 0,  0, L4, L5,  0,  0]
    result = [0] * 4
    result[0] = L0
    result[1] = L1 ^ L2
    result[2] = L3 ^ L4
    result[3] = L5

    # Mask each limb to 64 bits
    mask64 = (1 << 64) - 1
    for i in range(4):
        result[i] &= mask64

    return result

In [40]:
# Multiply two 128-bit polynomials
a = 123  # 128-bit input A
b = 123  # 128-bit input B

result = karatsuba_mul128(a, b)
print("Result limbs (low to high):", result)

Result limbs (low to high): [5445, 0, 0, 0]
